## Utils

### Imports

In [1]:
import requests
from environments.alfworld.common.environment import ALFWorldEnv

import os
from langchain_openai import ChatOpenAI
from planning_library.strategies import ADaPTStrategy
from planning_library.strategies.adapt.components import ADaPTPlanner, ADaPTExecutor
from planning_library.action_executors import GymnasiumActionExecutor
from textwrap import dedent

%load_ext autoreload
%autoreload 2

### Logging

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ALFWorld + ADaPT (new)"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

## Environment

In [3]:
os.environ["ALFWORLD_DATA"] = "utils/data"

if not "base_config.yaml" in os.listdir("utils"):
    response = requests.get("https://raw.githubusercontent.com/alfworld/alfworld/master/configs/base_config.yaml")
    
    with open("utils/base_config.yaml", "wb") as f:
        f.write(response.content)

env = ALFWorldEnv(config_path="utils/base_config.yaml")

Initializing AlfredTWEnv...


100%|██████████| 8810/8810 [00:03<00:00, 2827.99it/s]

Overall we have 3553 games in split=train
Training with 3553 games


In [4]:
obs, info = env.reset(options={"next_episode": True})
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.


## Strategy

### Hyperparameters

In [5]:
# ADaPT hyperparameters
max_depth = 3  # maximum decomposition depth
executor_max_iterations = 2  # maximum number of iterations for the executor; when reached, the loop will exit
planner_max_iterations = 20  # maximum number of iterations for the planner; when reached, the loop will exit

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

### Executor

Default Executor is powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).

In [6]:
action_executor = GymnasiumActionExecutor(env)
executor = ADaPTExecutor.create_simple_strategy(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                                tools=env.tools,
                                             parser_name="openai-tools", 
                                                action_executor=action_executor,
                                                user_message=dedent("""
            You are ALFRED, an intelligent agent navigating in a household.

            {inputs}"""),      max_iterations=executor_max_iterations,
                                                return_intermediate_steps=True,
                                                return_finish_log=True)

### Planner

Planner can either be:

* `agent`: llm with tools, can be powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).
* `simple`: a single call to llm.

In [7]:
agent_planner = ADaPTPlanner.create_agent_planner(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                             parser_name="openai-tools", 
                                                user_message=dedent("""
            Here is the essence of the original task:

            {inputs}
            """),
            executor_parser_name="openai-tools")

In [7]:
simple_planner = ADaPTPlanner.create_simple_planner(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                                user_message=dedent("""
            Here is the essence of the original task:

            {inputs}
            """),
            executor_parser_name="openai-tools")

## Defining strategy

In [9]:
agent_planner_adapt = ADaPTStrategy(
    executor=executor,
    planner=agent_planner,
    max_depth=max_depth,
    return_intermediate_steps=True,
    return_finish_log=True,
)

In [10]:
agent_planner_adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
I have constructed a step-by-step plan with an "and" aggregation logic to successfully solve the task of putting a pencil on the shelf:

1. Check if there is a pencil on the desk 1.
2. Put the pencil from desk 1 onto shelf 1.

The plan requires the successful completion of both subtasks in order to achieve the goal. Feel free to proceed with the execution of the plan.

> Finished chain.


> Entering new SimpleStrategy chain...
Task completed.

> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
I have formulated a step-by-step plan to solve the task successfully:
1. Take the pencil from desk 1.
2. Put the pencil onto shelf 1.

The plan follows the logic that all subtasks must be completed successfully in order to consider the task solved.

> Finished chain.


> Entering new Simpl

{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[(OpenAIToolAgentAction(tool='look', tool_input={}, log='\nInvoking: `look` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_2r8sx6QR3ZgHeW0O8MlO4a8e', 'function': {'arguments': '{}', 'name': 'look'}, 'type': 'function'}]})], tool_call_id='call_2r8sx6QR3ZgHeW0O8MlO4a8e'),
    {'observation': 'You are facing the cabinet 3, and shelf 1. Next to it, you see nothing.',
     'reward': 0,
     'terminated': False,
     'truncated': False,
     'info': {}}),
   (OpenAIToolAgentAction(tool='look', tool_input={}, log='\nInvoking: `look` with `{}`\n\n\n', message_log=[AIMessageChunk(content

In [8]:
simple_planner_adapt = ADaPTStrategy(
    executor=executor,
    planner=simple_planner,
    max_depth=max_depth,
    return_intermediate_steps=True,
    return_finish_log=True,
)

In [9]:
simple_planner_adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
It seems I couldn't go to shelf 1. Let me try again.

> Finished chain.


> Entering new SimpleStrategy chain...


> Finished chain.
Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Maximum decomposition depth reached.

> Finished chain.


{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[]],
 'finish_log': ["Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Couldn't solve the task. Last log: Maximum decomposition depth reached."]}